In [9]:
api_key = 'd2m9nr9r01qq6fopl8e0d2m9nr9r01qq6fopl8eg'

In [10]:
import finnhub
finnhub_client = finnhub.Client(api_key=api_key)

print(finnhub_client.news_sentiment('AAPL'))

FinnhubAPIException: FinnhubAPIException(status_code: 403): You don't have access to this resource.

In [8]:
print(finnhub_client.press_releases('AAPL'))

FinnhubAPIException: FinnhubAPIException(status_code: 403): You don't have access to this resource.

In [3]:
import requests
from datetime import date, datetime
from typing import List, Dict, Union, Optional
import time

def get_daily_news(ticker: str, day: Union[str, date, datetime], token: Optional[str] = None,
                   max_retries: int = 3, timeout: int = 10) -> List[Dict]:
    """
    Fetch Finnhub company news for a single ticker for a specific calendar day.

    Parameters
    ----------
    ticker : str
        Stock symbol, e.g. 'AAPL'.
    day : str | date | datetime
        Target day. If str, must be 'YYYY-MM-DD'.
    token : str | None
        Finnhub API token. If None, uses global api_key.
    max_retries : int
        Retry attempts on transient HTTP errors (429, 5xx).
    timeout : int
        Per-request timeout in seconds.

    Returns
    -------
    List[Dict]
        List of news item JSON objects (empty list if none or failure).

    Notes
    -----
    Finnhub endpoint: /company-news (no pagination). The 'from' and 'to' must be the same
    for a single day window.
    """
    if isinstance(day, (date, datetime)):
        day_str = day.strftime('%Y-%m-%d')
    else:
        day_str = day
    token = token or api_key  # uses existing variable from previous cell

    url = "https://finnhub.io/api/v1/company-news"
    params = {"symbol": ticker.upper(), "from": day_str, "to": day_str, "token": token}

    backoff = 1
    for attempt in range(1, max_retries + 1):
        try:
            r = requests.get(url, params=params, timeout=timeout)
            if r.status_code == 200:
                data = r.json()
                # Ensure it's a list
                return data if isinstance(data, list) else []
            if r.status_code == 429:
                # Rate limited: simple exponential backoff
                time.sleep(backoff)
                backoff = min(backoff * 2, 16)
                continue
            if 500 <= r.status_code < 600:
                continue  # retry on server errors
            # Non-retriable error
            return []
        except (requests.Timeout, requests.ConnectionError):
            continue
    return []

In [5]:
# Example (commented):
news_items = get_daily_news("AAPL", "2024-08-02")
print(len(news_items), "articles")

0 articles
